In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
len(coordinates)

1500

In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
len(cities)

616

In [8]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [11]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [12]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | tasiilaq
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | cidreira
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | provideniya
Processing Record 7 of Set 1 | nanortalik
Processing Record 8 of Set 1 | punta arenas
Processing Record 9 of Set 1 | havre-saint-pierre
Processing Record 10 of Set 1 | muravlenko
Processing Record 11 of Set 1 | grindavik
Processing Record 12 of Set 1 | bakel
Processing Record 13 of Set 1 | constitucion
Processing Record 14 of Set 1 | talnakh
Processing Record 15 of Set 1 | bereda
Processing Record 16 of Set 1 | bireun
Processing Record 17 of Set 1 | hilo
Processing Record 18 of Set 1 | thompson
Processing Record 19 of Set 1 | lebu
Processing Record 20 of Set 1 | olafsvik
Processing Record 21 of Set 1 | bluff
Processing Record 22 of Set 1 | mataura
Processing Record 23 of Set 1 | wuwei
Processing Record 24 of Set 1 | hermanus
Processing Record 25 of Se

Processing Record 38 of Set 4 | beringovskiy
Processing Record 39 of Set 4 | sisimiut
Processing Record 40 of Set 4 | nizhneyansk
City not found. Skipping...
Processing Record 41 of Set 4 | kruisfontein
Processing Record 42 of Set 4 | yulara
Processing Record 43 of Set 4 | amderma
City not found. Skipping...
Processing Record 44 of Set 4 | ixtapa
Processing Record 45 of Set 4 | portland
Processing Record 46 of Set 4 | balugaon
Processing Record 47 of Set 4 | bernau
Processing Record 48 of Set 4 | katsuura
Processing Record 49 of Set 4 | manokwari
Processing Record 50 of Set 4 | bell ville
Processing Record 1 of Set 5 | lazaro cardenas
Processing Record 2 of Set 5 | gat
Processing Record 3 of Set 5 | ebeltoft
Processing Record 4 of Set 5 | san patricio
Processing Record 5 of Set 5 | beyneu
Processing Record 6 of Set 5 | sao jose da coroa grande
Processing Record 7 of Set 5 | hay river
Processing Record 8 of Set 5 | aloleng
Processing Record 9 of Set 5 | chimbote
Processing Record 10 of 

Processing Record 29 of Set 8 | kadhan
Processing Record 30 of Set 8 | vaxjo
Processing Record 31 of Set 8 | nargana
Processing Record 32 of Set 8 | mbuji-mayi
Processing Record 33 of Set 8 | hohhot
Processing Record 34 of Set 8 | alice springs
Processing Record 35 of Set 8 | mbandaka
Processing Record 36 of Set 8 | daoukro
Processing Record 37 of Set 8 | praia da vitoria
Processing Record 38 of Set 8 | sacama
Processing Record 39 of Set 8 | tapes
Processing Record 40 of Set 8 | tabory
Processing Record 41 of Set 8 | shimoda
Processing Record 42 of Set 8 | pisco
Processing Record 43 of Set 8 | butaritari
Processing Record 44 of Set 8 | andevoranto
City not found. Skipping...
Processing Record 45 of Set 8 | turbat
Processing Record 46 of Set 8 | kiunga
Processing Record 47 of Set 8 | dillon
Processing Record 48 of Set 8 | kuopio
Processing Record 49 of Set 8 | tidore
City not found. Skipping...
Processing Record 50 of Set 8 | morondava
Processing Record 1 of Set 9 | kahului
Processing R

Processing Record 16 of Set 12 | wajima
Processing Record 17 of Set 12 | coihaique
Processing Record 18 of Set 12 | ano mera
Processing Record 19 of Set 12 | flinders
Processing Record 20 of Set 12 | bilma
Processing Record 21 of Set 12 | nantucket
Processing Record 22 of Set 12 | cam ranh
Processing Record 23 of Set 12 | novosheshminsk
Processing Record 24 of Set 12 | yeppoon
Processing Record 25 of Set 12 | kendari
Processing Record 26 of Set 12 | manta
Processing Record 27 of Set 12 | flin flon
Processing Record 28 of Set 12 | avera
Processing Record 29 of Set 12 | port macquarie
Processing Record 30 of Set 12 | dasoguz
Processing Record 31 of Set 12 | aksha
Processing Record 32 of Set 12 | labuhan
Processing Record 33 of Set 12 | mackenzie
Processing Record 34 of Set 12 | vianopolis
Processing Record 35 of Set 12 | komsomolskiy
Processing Record 36 of Set 12 | wulanhaote
City not found. Skipping...
Processing Record 37 of Set 12 | yermakovskoye
Processing Record 38 of Set 12 | gazl

In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tasiilaq,65.6145,-37.6368,37.56,97,100,10.25,GL,2022-04-16 04:05:04
1,Rikitea,-23.1203,-134.9692,77.76,71,1,14.12,PF,2022-04-16 04:07:28
2,Albany,42.6001,-73.9662,57.27,36,78,1.01,US,2022-04-16 04:05:13
3,Cidreira,-30.1811,-50.2056,64.36,67,84,7.18,BR,2022-04-16 03:58:39
4,Ushuaia,-54.8000,-68.3000,38.28,88,74,4.74,AR,2022-04-16 04:03:38
5,Provideniya,64.3833,-173.3000,32.65,95,100,22.10,RU,2022-04-16 04:07:28
6,Nanortalik,60.1432,-45.2371,27.34,80,91,7.34,GL,2022-04-16 03:58:31
7,Punta Arenas,-53.1500,-70.9167,42.91,70,0,4.61,CL,2022-04-16 04:04:34
8,Havre-Saint-Pierre,50.2334,-63.5986,33.15,99,100,3.42,CA,2022-04-16 04:07:29
9,Muravlenko,63.7898,74.5230,32.70,100,100,9.33,RU,2022-04-16 04:07:29


In [16]:
city_data_df = city_data_df[["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]]

In [17]:
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Tasiilaq,GL,2022-04-16 04:05:04,65.6145,-37.6368,37.56,97,100,10.25
1,Rikitea,PF,2022-04-16 04:07:28,-23.1203,-134.9692,77.76,71,1,14.12
2,Albany,US,2022-04-16 04:05:13,42.6001,-73.9662,57.27,36,78,1.01
3,Cidreira,BR,2022-04-16 03:58:39,-30.1811,-50.2056,64.36,67,84,7.18
4,Ushuaia,AR,2022-04-16 04:03:38,-54.8000,-68.3000,38.28,88,74,4.74
5,Provideniya,RU,2022-04-16 04:07:28,64.3833,-173.3000,32.65,95,100,22.10
6,Nanortalik,GL,2022-04-16 03:58:31,60.1432,-45.2371,27.34,80,91,7.34
7,Punta Arenas,CL,2022-04-16 04:04:34,-53.1500,-70.9167,42.91,70,0,4.61
8,Havre-Saint-Pierre,CA,2022-04-16 04:07:29,50.2334,-63.5986,33.15,99,100,3.42
9,Muravlenko,RU,2022-04-16 04:07:29,63.7898,74.5230,32.70,100,100,9.33


In [19]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")